In [ ]:
# Use Python 3.12 enviroment
%pip install pillow mutagen librosa soundfile numpy

In [ ]:
import base64
import shutil
import subprocess
from io import BytesIO
from pathlib import Path
from PIL import Image
from mutagen.oggvorbis import OggVorbis
from mutagen.flac import Picture
import librosa
import numpy as np

# -------------------------------------------------------------
# CONFIGURACIÓN DE DIRECTORIOS
# -------------------------------------------------------------
script_dir = Path().cwd()
input_dir = script_dir / "download_temp"
output_dir = script_dir.parent / "Assets" / "StreamingAssets" / "Music"

input_dir.mkdir(exist_ok=True)
output_dir.mkdir(parents=True, exist_ok=True)

# -------------------------------------------------------------
# 1. DESCARGAR PLAYLIST (yt-dlp)
# -------------------------------------------------------------
PLAYLIST_URL = "https://music.youtube.com/playlist?list=PL2SZb_BNUE8gqEwA_-QD8ym6BtURsRH2W&si=ZMU0aeUJm59TaFsm"

yt_dlp_command = [
    "yt-dlp",
    PLAYLIST_URL,
    "--format", "bestaudio",
    "--extract-audio",
    "--audio-format", "vorbis",
    "--postprocessor-args", "ffmpeg:-qscale:a 8",
    "--embed-metadata",
    "--add-metadata",
    "--embed-thumbnail",
    "--convert-thumbnails", "jpg",
    "-o", f"{input_dir}/%(title)s [%(id)s].%(ext)s"
]

print("Descargando playlist...\n")
subprocess.run(yt_dlp_command, check=True)
print("Descarga completada.\n")

# -------------------------------------------------------------
# TOOLS
# -------------------------------------------------------------

def make_square(img):
    w, h = img.size
    if w == h:
        return img
    side = min(w, h)
    left = (w - side) // 2
    top = (h - side) // 2
    return img.crop((left, top, left + side, top + side))

def calculate_bpm(path):
    try:
        y, sr = librosa.load(path, sr=None, mono=True)
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
        return int(round(float(tempo)))
    except Exception as e:
        print("Error BPM:", e)
        return None

def detect_key(y, sr):
    try:
        chroma = librosa.feature.chroma_cqt(y=y, sr=sr)
        chroma_avg = chroma.mean(axis=1)
        note_names = ["C","C#","D","D#","E","F","F#","G","G#","A","A#","B"]
        idx = np.argmax(chroma_avg)
        return note_names[idx]
    except:
        return None

camelot_map = {
    "C":  "8B", "C#": "3B", "D":  "10B", "D#": "5B", "E":  "12B", "F":  "7B",
    "F#": "2B", "G":  "9B",  "G#": "4B", "A":  "11B","A#": "6B", "B":  "1B"
}

def compute_replaygain(path):
    try:
        y, sr = librosa.load(path, sr=None, mono=True)
        y = y / np.max(np.abs(y))

        loudness = np.mean(20*np.log10(np.abs(y) + 1e-6))
        track_gain = -18 - loudness  # target RG: −18 LUFS

        peak = float(np.max(np.abs(y)))
        return f"{track_gain:.2f} dB", f"{peak:.6f}"
    except Exception as e:
        print("ReplayGain error:", e)
        return None, None

# -------------------------------------------------------------
# PROCESAMIENTO COMPLETO POR ARCHIVO
# -------------------------------------------------------------
def process_file(src_path):
    dst_path = output_dir / src_path.name
    shutil.copy2(src_path, dst_path)

    audio = OggVorbis(dst_path)

    # --------------------
    # BPM
    # --------------------
    bpm = calculate_bpm(dst_path)
    if bpm:
        audio["BPM"] = str(bpm)

    # --------------------
    # KEY + CAMELOT
    # --------------------
    try:
        y, sr = librosa.load(dst_path, sr=None, mono=True)
        note = detect_key(y, sr)
        if note:
            audio["INITIALKEY"] = note
            audio["CAMELOT"] = camelot_map.get(note, "")
    except:
        pass

    # --------------------
    # REPLAYGAIN
    # --------------------
    gain, peak = compute_replaygain(dst_path)
    if gain and peak:
        audio["REPLAYGAIN_TRACK_GAIN"] = gain
        audio["REPLAYGAIN_TRACK_PEAK"] = peak

    # --------------------
    # PORTADA CUADRADA
    # --------------------
    if "metadata_block_picture" in audio:
        pic_b64 = audio["metadata_block_picture"][0]
        raw = base64.b64decode(pic_b64)

        pic = Picture(raw)
        img = Image.open(BytesIO(pic.data)).convert("RGB")
        img_sq = make_square(img)

        buf = BytesIO()
        img_sq.save(buf, format="JPEG", quality=97)
        jpg_bytes = buf.getvalue()

        newpic = Picture()
        newpic.type = 3
        newpic.mime = "image/jpeg"
        newpic.desc = ""
        newpic.width, newpic.height = img_sq.size
        newpic.depth = 24
        newpic.data = jpg_bytes

        encoded = base64.b64encode(newpic.write()).decode("ascii")
        audio["metadata_block_picture"] = [encoded]

    audio.save()


# -------------------------------------------------------------
# PROCESAR TODO LO DESCARGADO
# -------------------------------------------------------------
for f in input_dir.iterdir():
    if f.suffix.lower() == ".ogg":
        print("Procesando:", f.name)
        process_file(f)

# Limpieza
for f in input_dir.iterdir():
    if f.is_file():
        f.unlink()

print("\n✔ Procesamiento finalizado.")
print("Archivos listos en:", output_dir)

Procesamiento finalizado. Archivos en: c:\Users\Jvnc\Documents\IHC\DJ-Game\Assets\StreamingAssets\Music
